# Taylor Polynomial Approximations

Adam Rumpf

Created 4/13/21

Based on a previous [Mathematica class demonstration](https://github.com/adam-rumpf/mathematica-class-demonstrations#taylor-and-fourier-series-approximations).

Find more Jupyter Notebook class demonstrations [here](https://github.com/adam-rumpf/jupyter-class-demonstrations).

## Introduction

This demonstration defines two Manipulate environments to explore the construction of Taylor series approximation. It includes a dropdown menu to select an example function and a slider to select a number of terms. The display will show a plot of the function along with its approximation of the specified degree. The center of the approximation can be chosen using a slider.

The purpose of this demonstration is to illustrate the effects of adding more terms to the approximation, and how in general this causes the approximation to be closer to the original function over a larger area. Unusual features, like discontinuity, may cause problems for the Taylor approximation, but less of a problem for the Fourier approximation. The Taylor approximation is generally best near its center.

**Outline**

* Introducing problem of estimating a function value as closely as possible near $a$ using a limited-degree polynomial.
  * Still images of good polynomial approximations.
* For $n=0$, the only real choice is to match $p(a) = f(a)$.
  * Playing with an example.
* Explaining linearization, which is a major and useful concept on its own, and how we define the tangent line in caclulus.
  * Playing with an example of linearization.
* Extending to quadratic approximations and the $\frac{1}{2}$ factor.
  * Playing with an example of quadratic approximations.
* Building up the approximations to explain where the $n!$ divisor comes from.
* Defining Taylor polynomials.
  * Playing with Taylor approximations.
* Alluding to some examples of what Taylor polynomials are used for, how the error term works, Taylor series, and intervals of convergence.

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Define function derivatives
def f0(x):
    return np.sin(x)
def f1(x):
    return np.cos(x)
def f2(x):
    return -np.sin(x)
def f3(x):
    return -np.cos(x)
def g0(x):
    return np.exp(x)

# Define function lists
func = [[f0, f1, f2, f3]*2, [f1, f2, f3, f0]*2, [g0]*8]
fname = ["sin(x)", "cos(x)", "exp(x)"]

# Define Taylor polynomial
def taylor(x, fid, a, n):
    """Taylor polynomial for a given function.
    
    Positional arguments:
    x - input value
    fid - function ID from 'func' list
    a - center
    n - polynomial degree
    """
    
    global func
    
    out = 0.0 # output value
    
    # Add terms of Taylor polynomial
    for i in range(n+1):
        out += (func[fid][i](a)/np.math.factorial(i))*(x - a)**i
    
    return out

# Set up plot
fig, ax = plt.subplots()

# Generate x values
x = np.linspace(-2*np.pi, 2*np.pi, 101)
y = np.zeros_like(x) # initialize Taylor output array

# Draw plot lines
@widgets.interact(a=(-2*np.pi, 2*np.pi, 0.05), f=(0, 2, 1), n=(0, 7, 1))
def update1(a=0.0, f=0, n=1):
    global ax, y
    ax.clear()
    ax.set_ylim([-4, 4])
    plt.title(fname[f])
    ax.grid(True)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.plot(x, func[f][0](x), color="C0")
    y = taylor(x, f, a, n)
    ax.plot(x, y, color="C1")
    ax.plot(a, func[f][0](a), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='a', max=6.283185307179586, min=-6.283185307179586, s…